<a href="https://colab.research.google.com/github/tohsin/Deeplearning/blob/master/same_model_and_larger_images.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np
import os
import shutil
from keras.preprocessing.image import ImageDataGenerator
from tensorflow import keras
from keras.applications.densenet import DenseNet121
from keras.layers import Input
from keras.applications import DenseNet121
from keras import models,regularizers
from keras import layers
from keras.layers import Dense
import matplotlib.pyplot as plt

In [ ]:
!unzip "/content/drive/My Drive/Colab Notebooks/chexnet/train.zip"

In [ ]:
df=pd.read_csv('/content/drive/My Drive/Colab Notebooks/chexnet/Train.csv')
df=df.drop("ID",axis=1)
df.head()

,filename,LABEL
0,train/VTYNBFTO.png,1
1,train/GVVXXDOZ.png,0
2,train/PMRSQLXN.png,0
3,train/JXISWDUU.png,1
4,train/JVWMAACC.png,1


Create dir for train ,test ,split

In [ ]:
root_dir='/train'
root_dirs='/content/drive/My Drive/Colab Notebooks/chexnet/data'
classes_dir = ['1', '0']

train_ratio=0.7
val_ratio = 0.15
test_ratio = 0.15

os.makedirs(root_dirs+'/train' ,exist_ok=True)
os.makedirs(root_dirs +'/val' ,exist_ok=True)
os.makedirs(root_dirs +'/test',exist_ok=True )


for cls in classes_dir:
    train_dir = os.path.join(root_dirs+'/train', cls) 
    os.mkdir(train_dir)
    test_dir = os.path.join(root_dirs+'/test', cls) 
    os.mkdir(test_dir)
    val_dir = os.path.join(root_dirs+'/val', cls) 
    os.mkdir(val_dir)
    

    # Creating partitions of the data after shuffeling
    allFileNames=df['filename'][df['LABEL']==int(cls)].tolist() # Folder to copy images from
    
    np.random.shuffle(allFileNames)
    train_FileNames=allFileNames[:int(len(allFileNames) *train_ratio)]
    val_FileNames=allFileNames[len(train_FileNames) :len(train_FileNames)+int((len(allFileNames)*val_ratio))]
    test_FileNames=allFileNames[len(train_FileNames) +len(val_FileNames):]




    print('Total images: ', len(allFileNames))
    print('Training: ', len(train_FileNames))
    print('Validation: ', len(val_FileNames))
    print('Testing: ', len(test_FileNames))

    # Copy-pasting images
    count=0
    for name in train_FileNames:
        shutil.copy('/content'+'/'+name, train_dir)
        count+=1
    print(count)

    for name in val_FileNames:
        shutil.copy('/content'+'/'+name, val_dir)

    for name in test_FileNames:
        shutil.copy('/content'+'/'+name,test_dir)

Total images:  353
Training:  247
Validation:  52
Testing:  54
247
Total images:  365
Training:  255
Validation:  54
Testing:  56
255


In [ ]:
root_dirs='/content/drive/My Drive/Colab Notebooks/chexnet/data'
train_datagen = ImageDataGenerator(
    rescale=1./255,
          featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range = 30,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.2, # Randomly zoom image 
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip = True,  # randomly flip images
        vertical_flip=False)  # randomly flip images


test_datagen = ImageDataGenerator(rescale=1./255)



# train_datagen = ImageDataGenerator(
#   featurewise_center=True,
#   featurewise_std_normalization=True,
#   rotation_range=10,
#   width_shift_range=0,
#   height_shift_range=0,
#   vertical_flip=False,)





train_generator = train_datagen.flow_from_directory(
    root_dirs+'/train',
    batch_size=16,
    target_size=(224, 224),
    class_mode='binary')
validation_generator = test_datagen.flow_from_directory( 
    root_dirs+'/val',
    target_size=(224, 224),
    batch_size=16,
    class_mode='binary')

Found 502 images belonging to 2 classes.
Found 106 images belonging to 2 classes.


In [ ]:
conv_base=DenseNet121(
    include_top=False,
    weights="imagenet",
    input_shape=(224,224,3),
)
flat = keras.layers.Flatten()(conv_base.output) 
l1 = keras.layers.Dense(52, activation="relu")(flat) 
l2 = keras.layers.Dropout(0.3)(l1) 
output = keras.layers.Dense(1, activation="sigmoid")(l2) 
model = keras.models.Model(inputs=conv_base.input, outputs=output)
for layer in conv_base.layers:
   layer.trainable = False

29089792/29084464 [==============================] - 0s 0us/step


In [ ]:
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
zero_padding2d (ZeroPadding2D)  (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1/conv (Conv2D)             (None, 112, 112, 64) 9408        zero_padding2d[0][0]             
__________________________________________________________________________________________________
conv1/bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1/conv[0][0]                 
_______________________________________________________________________________________

In [ ]:
checkpoint_filepath =   '/content/drive/My Drive/Colab Notebooks/chexnet/tmp/checkpoint/lhead.h5'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=False,
    save_freq="epoch",
    monitor='val_acc',
    mode='max',
    verbose=1,
    save_best_only=True)
dataset_size=502
optimizer1=tf.keras.optimizers.Adam(
    learning_rate= 0.001,  name="Adamish",decay=0.001
)
model.compile(loss='binary_crossentropy', 
              optimizer=optimizer1,
              metrics=['acc'])

history = model.fit(train_generator,
      steps_per_epoch=502//16,
      callbacks=[model_checkpoint_callback],
      epochs=30,
      validation_data=validation_generator,)

Epoch 1/30
31/31 [==============================] - ETA: 0s - loss: 0.9306 - acc: 0.6276
Epoch 00001: val_acc improved from -inf to 0.52830, saving model to /content/drive/My Drive/Colab Notebooks/chexnet/data/tmp/checkpoint/lhead.h5
31/31 [==============================] - 163s 5s/step - loss: 0.9306 - acc: 0.6276 - val_loss: 0.6523 - val_acc: 0.5283
Epoch 2/30
31/31 [==============================] - ETA: 0s - loss: 0.5617 - acc: 0.7119
Epoch 00002: val_acc improved from 0.52830 to 0.77358, saving model to /content/drive/My Drive/Colab Notebooks/chexnet/data/tmp/checkpoint/lhead.h5
31/31 [==============================] - 146s 5s/step - loss: 0.5617 - acc: 0.7119 - val_loss: 0.5380 - val_acc: 0.7736
Epoch 3/30
31/31 [==============================] - ETA: 0s - loss: 0.5605 - acc: 0.7325
Epoch 00003: val_acc did not improve from 0.77358
31/31 [==============================] - 147s 5s/step - loss: 0.5605 - acc: 0.7325 - val_loss: 0.6925 - val_acc: 0.5849
Epoch 4/30
31/31 [============

In [ ]:
model.trainable=True
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
zero_padding2d (ZeroPadding2D)  (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1/conv (Conv2D)             (None, 112, 112, 64) 9408        zero_padding2d[0][0]             
__________________________________________________________________________________________________
conv1/bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1/conv[0][0]                 
_______________________________________________________________________________________

In [ ]:
model.load_weights('/content/drive/My Drive/Colab Notebooks/chexnet/tmp/checkpoint/lhead.h5')

In [ ]:
checkpoint_filepath = '/content/drive/My Drive/Colab Notebooks/chexnet/tmp/checkpoint/lhead_2.h5'
optimizer=tf.keras.optimizers.Adam(
    learning_rate= 0.001,  name="Adamish",decay=0.001
)
model_checkpoint_callback_ = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=False,
    save_freq="epoch",
    monitor='val_acc',
    mode='max',
    verbose=1,
    save_best_only=True)

model.compile(loss='binary_crossentropy', 
              optimizer=optimizer,
              metrics=['acc'])

history = model.fit(train_generator,
      steps_per_epoch=502//16,
      callbacks=[model_checkpoint_callback_],
      epochs=90,
      validation_data=validation_generator,)

model.load_weights(checkpoint_filepath)

Epoch 1/90
31/31 [==============================] - ETA: 0s - loss: 0.5769 - acc: 0.7325
Epoch 00001: val_acc improved from -inf to 0.82075, saving model to /content/drive/My Drive/Colab Notebooks/chexnet/tmp/checkpoint/lhead_2.h5
31/31 [==============================] - 163s 5s/step - loss: 0.5769 - acc: 0.7325 - val_loss: 0.5455 - val_acc: 0.8208
Epoch 2/90
31/31 [==============================] - ETA: 0s - loss: 0.5829 - acc: 0.7037
Epoch 00002: val_acc did not improve from 0.82075
31/31 [==============================] - 153s 5s/step - loss: 0.5829 - acc: 0.7037 - val_loss: 0.5428 - val_acc: 0.8019
Epoch 3/90
31/31 [==============================] - ETA: 0s - loss: 0.5609 - acc: 0.7099
Epoch 00003: val_acc did not improve from 0.82075
31/31 [==============================] - 154s 5s/step - loss: 0.5609 - acc: 0.7099 - val_loss: 0.5359 - val_acc: 0.8019
Epoch 4/90
31/31 [==============================] - ETA: 0s - loss: 0.5683 - acc: 0.7222
Epoch 00004: val_acc did not improve from 